This notebook contains an example for teaching.


# Automatic Machine Learning with H2O AutoML using Wage Data from 2015

We illustrate how to predict an outcome variable Y in a high-dimensional setting, using the AutoML package *H2O* that covers the complete pipeline from the raw dataset to the deployable machine learning model. In last few years, AutoML or automated machine learning has become widely popular among data science community. 

We can use AutoML as a benchmark and compare it to the methods that we used in the previous notebook where we applied one machine learning method after the other.

This notebook contains an example for teaching.


# Automatic Machine Learning with H2O AutoML using Wage Data from 2015

We illustrate how to predict an outcome variable Y in a high-dimensional setting, using the AutoML package *H2O* that covers the complete pipeline from the raw dataset to the deployable machine learning model. In last few years, AutoML or automated machine learning has become widely popular among data science community. 

We can use AutoML as a benchmark and compare it to the methods that we used in the previous notebook where we applied one machine learning method after the other.

In [1]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 15.0.2+7-27, mixed mode, sharing)
  Starting server from C:\Users\MSI-NB\anaconda3\envs\tensorflow_2\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\MSI-NB\AppData\Local\Temp\tmpvqdly921
  JVM stdout: C:\Users\MSI-NB\AppData\Local\Temp\tmpvqdly921\h2o_MSI_NB_started_from_python.out
  JVM stderr: C:\Users\MSI-NB\AppData\Local\Temp\tmpvqdly921\h2o_MSI_NB_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Lima
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_MSI_NB_quco3q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.965 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"
